In [1]:
!pip install "ray[tune]" optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 KB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 KB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.5/468.5 KB 20.2 MB/s eta 0:00:00


In [2]:
#noraml optuna tuning
import torch
import torch.optim as optim
from ray.tune.examples.mnist_pytorch import get_data_loaders, ConvNet, train, test
import optuna


def train_mnist(trial):
   use_cuda = torch.cuda.is_available()
   device = torch.device("cuda" if use_cuda else "cpu")
   train_loader, test_loader = get_data_loaders()
   model = ConvNet().to(device)

   optimizer = optim.SGD(
       model.parameters(),
       lr=trial.suggest_loguniform("lr", 1e-4, 1e-2),
       momentum=trial.suggest_uniform("momentum", 0.1, 0.9))

   for i in range(20):
       train(model, optimizer, train_loader, device)
       acc = test(model, test_loader, device)
       trial.report(-acc, i)
   return -acc


import time
start = time.time()
study = optuna.create_study()
study.optimize(train_mnist, n_jobs=1, n_trials=10)
taken = time.time() - start
print(f"Time taken: {taken:.2f} seconds.")
print(f"Best config: {study.best_params}")

[I 2023-03-31 00:47:16,952] A new study created in memory with name: no-name-1f0f8195-d28a-4cf4-9c2f-7f45a4888003


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting /root/data/MNIST/raw/train-images-idx3-ubyte.gz to /root/data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting /root/data/MNIST/raw/train-labels-idx1-ubyte.gz to /root/data/MNIST/raw



...

Extracting /root/data/MNIST/raw/t10k-images-idx3-ubyte.gz to /root/data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting /root/data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /root/data/MNIST/raw



<ipython-input-2-720b16afea0b>:16: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  lr=trial.suggest_loguniform("lr", 1e-4, 1e-2),
<ipython-input-2-720b16afea0b>:17: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  momentum=trial.suggest_uniform("momentum", 0.1, 0.9))
[I 2023-03-31 00:47:27,388] Trial 0 finished with value: -0.825 and parameters: {'lr': 0.005368459171803371, 'momentum': 0.1747951191265278}. Best is trial 0 with value: -0.825.
[I 2023-03-31 00:47:33,668] Trial 1 finished with value: -0.753125 and parameters: {'lr': 0.0012665032269518304, 'momentum': 0.7855668160548126}. Best is trial 0 with value: -0.825.
[I 2023-03-31 00:47:41,942] Tria

Time taken: 75.36 seconds.
Best config: {'lr': 0.005368459171803371, 'momentum': 0.1747951191265278}


In [3]:
#Optuna +Ray tuning
import torch
import torch.optim as optim
from ray.tune.examples.mnist_pytorch import get_data_loaders, ConvNet, train, test
from ray import tune
from ray.tune.suggest.optuna import OptunaSearch


def train_mnist(config):
   use_cuda = torch.cuda.is_available()
   device = torch.device("cuda" if use_cuda else "cpu")
   train_loader, test_loader = get_data_loaders()
   model = ConvNet().to(device)

   optimizer = optim.SGD(
       model.parameters(), lr=config["lr"], momentum=config["momentum"])

   for i in range(20):
       train(model, optimizer, train_loader, device)
       acc = test(model, test_loader, device)
       tune.report(mean_accuracy=acc)


import time
start = time.time()
analysis = tune.run(
   train_mnist,
   config={
       "lr": tune.loguniform(1e-4, 1e-2),
       "momentum": tune.uniform(0.1, 0.9),
   },
   metric="mean_accuracy",
   mode="max",
   search_alg=OptunaSearch(),
   num_samples=10)
taken = time.time() - start
print(f"Time taken: {taken:.2f} seconds.")
print(f"Best config: {analysis.best_config}")


<ipython-input-3-05c7435b5f9b>:6: DeprecationWarning: The module `ray.tune.suggest` has been moved to `ray.tune.search` and the old location will be deprecated soon. Please adjust your imports to point to the new location. Example: Do a global search and replace `ray.tune.suggest` with `ray.tune.search`.
  from ray.tune.suggest.optuna import OptunaSearch
<ipython-input-3-05c7435b5f9b>:6: DeprecationWarning: The module `ray.tune.suggest.optuna` has been moved to `ray.tune.search.optuna` and the old location will be deprecated soon. Please adjust your imports to point to the new location. Example: Do a global search and replace `ray.tune.suggest.optuna` with `ray.tune.search.optuna`.
  from ray.tune.suggest.optuna import OptunaSearch
2023-03-31 00:48:34,418	INFO worker.py:1553 -- Started a local Ray instance.
/usr/local/lib/python3.9/dist-packages/ray/tune/search/optuna/optuna_search.py:664: FutureWarning: LogUniformDistribution has been deprecated in v3.0.0. This feature will be removed

/usr/local/lib/python3.9/dist-packages/optuna/distributions.py:799: FutureWarning: LogUniformDistribution(high=0.01, low=0.0001) is deprecated and internally converted to FloatDistribution(high=0.01, log=True, low=0.0001, step=None). See https://github.com/optuna/optuna/issues/2941.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.9/dist-packages/optuna/distributions.py:799: FutureWarning: UniformDistribution(high=0.9, low=0.1) is deprecated and internally converted to FloatDistribution(high=0.9, log=False, low=0.1, step=None). See https://github.com/optuna/optuna/issues/2941.
  warnings.warn(message, FutureWarning)


Trial name,date,done,episodes_total,experiment_id,experiment_tag,hostname,iterations_since_restore,mean_accuracy,node_ip,pid,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
train_mnist_219b2471,2023-03-31_00-49-08,True,,387a93cbe494420abb17b74ab0b2b266,"1_lr=0.0001,momentum=0.8722",f69258776f2b,20,0.221875,172.28.0.12,1003,18.4266,0.541825,18.4266,1680223748,0,,20,219b2471,0.00914192
train_mnist_6f26448b,2023-03-31_00-49-38,True,,387a93cbe494420abb17b74ab0b2b266,"5_lr=0.0020,momentum=0.2584",f69258776f2b,20,0.303125,172.28.0.12,1003,14.6371,0.559467,14.6371,1680223778,0,,20,6f26448b,0.00914192
train_mnist_8205f1ca,2023-03-31_00-49-09,True,,aec957653e17440682ae3933ff683097,"2_lr=0.0033,momentum=0.5468",f69258776f2b,20,0.840625,172.28.0.12,1095,14.662,0.636023,14.662,1680223749,0,,20,8205f1ca,0.00951219
train_mnist_c87f3c0b,2023-03-31_00-49-51,True,,387a93cbe494420abb17b74ab0b2b266,"7_lr=0.0059,momentum=0.3331",f69258776f2b,20,0.79375,172.28.0.12,1003,13.399,1.18657,13.399,1680223791,0,,20,c87f3c0b,0.00914192
train_mnist_c909c48b,2023-03-31_00-49-23,True,,387a93cbe494420abb17b74ab0b2b266,"3_lr=0.0010,momentum=0.2245",f69258776f2b,20,0.421875,172.28.0.12,1003,14.9478,0.542114,14.9478,1680223763,0,,20,c909c48b,0.00914192
train_mnist_e44df06b,2023-03-31_00-50-06,True,,aec957653e17440682ae3933ff683097,"10_lr=0.0049,momentum=0.2047",f69258776f2b,20,0.865625,172.28.0.12,1095,12.582,0.536664,12.582,1680223806,0,,20,e44df06b,0.00951219
train_mnist_e4c64342,2023-03-31_00-49-38,True,,aec957653e17440682ae3933ff683097,"6_lr=0.0006,momentum=0.7751",f69258776f2b,20,0.55625,172.28.0.12,1095,14.8726,0.525694,14.8726,1680223778,0,,20,e4c64342,0.00951219
train_mnist_ea6ddeea,2023-03-31_00-50-05,True,,387a93cbe494420abb17b74ab0b2b266,"9_lr=0.0006,momentum=0.6876",f69258776f2b,20,0.203125,172.28.0.12,1003,14.0184,0.940733,14.0184,1680223805,0,,20,ea6ddeea,0.00914192
train_mnist_f719cf6a,2023-03-31_00-49-53,True,,aec957653e17440682ae3933ff683097,"8_lr=0.0001,momentum=0.1026",f69258776f2b,20,0.08125,172.28.0.12,1095,14.844,1.72508,14.844,1680223793,0,,20,f719cf6a,0.00951219
train_mnist_fbba0fa4,2023-03-31_00-49-24,True,,aec957653e17440682ae3933ff683097,"4_lr=0.0010,momentum=0.1651",f69258776f2b,20,0.221875,172.28.0.12,1095,14.8018,0.602997,14.8018,1680223764,0,,20,fbba0fa4,0.00951219


2023-03-31 00:50:06,628	INFO tune.py:798 -- Total run time: 89.40 seconds (88.69 seconds for the tuning loop).


Time taken: 94.36 seconds.
Best config: {'lr': 0.004917195096708547, 'momentum': 0.20471080570263017}


In [5]:
ray.shutdown()
import ray

ray.init()

2023-03-31 02:33:39,984	INFO worker.py:1553 -- Started a local Ray instance.


Python version:,3.9.16
Ray version:,2.3.1
